In [1]:
"""testing the multidim wav2vec model against the reference model"""
import torch
import numpy as np
from lightning.pytorch import seed_everything
import seisbench.data as sbd

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers import Wav2Vec2Config
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining

data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms[0]).unsqueeze(0)


MODEL_NAMES = ["patrickvonplaten/wav2vec2-base-v2", "facebook/wav2vec2-base"]

model_output = {}
for model_name in MODEL_NAMES:
  config = Wav2Vec2Config.from_pretrained(model_name)

  for model_type in ['new']:

    seed_everything(0)
    if model_type == 'ref':
      model = RefWav2Vec2ForPreTraining(config)
    else:
      ref_model = RefWav2Vec2ForPreTraining(config)
      model = MultiDimWav2Vec2ForPreTraining(config)
      model.load_state_dict(ref_model.state_dict())
      del ref_model

    # compute masked indices
    batch_size, raw_sequence_length = input_values.shape
    sequence_length = model._get_feat_extract_output_lengths(
      raw_sequence_length).item()

    seed_everything(0)
    mask_time_indices = _compute_mask_indices(
        shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
    )
    sampled_negative_indices = _sample_negative_indices(
        features_shape=(batch_size, sequence_length),
        num_negatives=model.config.num_negatives,
        mask_time_indices=mask_time_indices,
    )
    mask_time_indices = torch.tensor(
      data=mask_time_indices, device=input_values.device, dtype=torch.long)
    sampled_negative_indices = torch.tensor(
      data=sampled_negative_indices, device=input_values.device,
      dtype=torch.long
    )

    with torch.no_grad():
      outputs = model(input_values, mask_time_indices=mask_time_indices,
                      sampled_negative_indices=sampled_negative_indices)

    print(outputs.codevector_perplexity)
    model_output[f'{model_name}_{model_type}'] = outputs



2024-06-15 20:17:16,740 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.
/scicore/home/dokman0000/liu0003/anaconda3/envs/seisbench/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Seed set to 0
Seed set to 0
/scicore/home/dokman0000/liu0003/anaconda3/envs/seisbench/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Seed set to 0


tensor(8.2678)


Seed set to 0


tensor(11.1942)


In [2]:
model_output = {}
for model_name in MODEL_NAMES:
  config = Wav2Vec2Config.from_pretrained(model_name)
  config.sinkhorn_quantization_iters = 1

  for model_type in ['new']:

    seed_everything(0)
    if model_type == 'ref':
      model = RefWav2Vec2ForPreTraining(config)
    else:
      ref_model = RefWav2Vec2ForPreTraining(config)
      model = MultiDimWav2Vec2ForPreTraining(config)
      model.load_state_dict(ref_model.state_dict())
      del ref_model

    # compute masked indices
    batch_size, raw_sequence_length = input_values.shape
    sequence_length = model._get_feat_extract_output_lengths(
      raw_sequence_length).item()

    seed_everything(0)
    mask_time_indices = _compute_mask_indices(
        shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
    )
    sampled_negative_indices = _sample_negative_indices(
        features_shape=(batch_size, sequence_length),
        num_negatives=model.config.num_negatives,
        mask_time_indices=mask_time_indices,
    )
    mask_time_indices = torch.tensor(
      data=mask_time_indices, device=input_values.device, dtype=torch.long)
    sampled_negative_indices = torch.tensor(
      data=sampled_negative_indices, device=input_values.device,
      dtype=torch.long
    )

    with torch.no_grad():
      outputs = model(input_values, mask_time_indices=mask_time_indices,
                      sampled_negative_indices=sampled_negative_indices)

    print(outputs.codevector_perplexity)
    model_output[f'{model_name}_{model_type}'] = outputs


Seed set to 0
Seed set to 0


EinopsError:  Error while processing rearrange-reduction pattern "(g b l) v -> b (l g) v".
 Input tensor shape: torch.Size([2, 18, 320]). Additional info: {'b': 1, 'l': 18, 'g': 2}.
 Wrong shape: expected 2 dims. Received 3-dim tensor.